In [0]:
import os
os._exit(0)

In [0]:
!ls drive/mentes

In [0]:
!dir

In [0]:
!fusermount -u drive

In [0]:
!fusermount -u drive
!rm -r *
#!find . ! -name 'drive' -type d -exec rm -d {} +
!dir
!free -g
#!cat /proc/meminfo

### Regi

In [0]:
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
# !add-apt-repository ppa:alessandro-strada/google-drive-ocamlfuse-beta

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

### uj  Fuse mounting

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive/mentes/trainer/

In [0]:
!ls drive

# CIFAR

In [0]:
#Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Functions for downloading and reading MNIST data (deprecated).
This module and all its submodules are deprecated. See
[contrib/learn/README.md](https://www.tensorflow.org/code/tensorflow/contrib/learn/README.md)
for migration instructions.
"""
from keras.datasets import cifar10
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf

import gzip

import numpy 
from six.moves import xrange  # pylint: disable=redefined-builtin

from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import random_seed
from tensorflow.python.platform import gfile
from tensorflow.python.util.deprecation import deprecated

# CVDF mirror of http://yann.lecun.com/exdb/mnist/
DEFAULT_SOURCE_URL = 'https://storage.googleapis.com/cvdf-datasets/mnist/'


def _read32(bytestream):
  dt = numpy.dtype(numpy.uint32).newbyteorder('>')
  return numpy.frombuffer(bytestream.read(4), dtype=dt)[0]

@deprecated(None, 'Please use tf.data to implement this functionality.')
def extract_images(f):
  """Extract the images into a 4D uint8 numpy array [index, y, x, depth].
  Args:
    f: A file object that can be passed into a gzip reader.
  Returns:
    data: A 4D uint8 numpy array [index, y, x, depth].
  Raises:
    ValueError: If the bytestream does not start with 2051.
  """
  print('Extracting', f.name)
  with gzip.GzipFile(fileobj=f) as bytestream:
    magic = _read32(bytestream)
    if magic != 2051:
      raise ValueError('Invalid magic number %d in MNIST image file: %s' %
                       (magic, f.name))
    num_images = _read32(bytestream)
    rows = _read32(bytestream)
    cols = _read32(bytestream)
    buf = bytestream.read(rows * cols * num_images)
    data = numpy.frombuffer(buf, dtype=numpy.uint8)
    data = data.reshape(num_images, rows, cols, 1)
    return data


@deprecated(None, 'Please use tf.one_hot on tensors.')
def dense_to_one_hot(labels_dense, num_classes):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = numpy.arange(num_labels) * num_classes
  labels_one_hot = numpy.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot


@deprecated(None, 'Please use tf.data to implement this functionality.')
def extract_labels(f, one_hot=False, num_classes=10):
  """Extract the labels into a 1D uint8 numpy array [index].
  Args:
    f: A file object that can be passed into a gzip reader.
    one_hot: Does one hot encoding for the result.
    num_classes: Number of classes for the one hot encoding.
  Returns:
    labels: a 1D uint8 numpy array.
  Raises:
    ValueError: If the bystream doesn't start with 2049.
  """
  print('Extracting', f.name)
  with gzip.GzipFile(fileobj=f) as bytestream:
    magic = _read32(bytestream)
    if magic != 2049:
      raise ValueError('Invalid magic number %d in MNIST label file: %s' %
                       (magic, f.name))
    num_items = _read32(bytestream)
    buf = bytestream.read(num_items)
    labels = numpy.frombuffer(buf, dtype=numpy.uint8)
    if one_hot:
      return dense_to_one_hot(labels, num_classes)
    return labels


class DataSet(object):
  """Container class for a dataset (deprecated).
  THIS CLASS IS DEPRECATED. See
  [contrib/learn/README.md](https://www.tensorflow.org/code/tensorflow/contrib/learn/README.md)
  for general migration instructions.
  """

  @deprecated(None, 'Please use alternatives such as official/mnist/dataset.py'
              ' from tensorflow/models.')
  def __init__(self,
               images,
               labels,
               fake_data=False,
               one_hot=False,
               dtype=dtypes.float32,
               reshape=True,
               seed=None):
    """Construct a DataSet.
    one_hot arg is used only if fake_data is true.  `dtype` can be either
    `uint8` to leave the input as `[0, 255]`, or `float32` to rescale into
    `[0, 1]`.  Seed arg provides for convenient deterministic testing.
    """
    seed1, seed2 = random_seed.get_seed(seed)
    # If op level seed is not set, use whatever graph level seed is returned
    numpy.random.seed(seed1 if seed is None else seed2)
    dtype = dtypes.as_dtype(dtype).base_dtype
    if dtype not in (dtypes.uint8, dtypes.float32):
      raise TypeError(
          'Invalid image dtype %r, expected uint8 or float32' % dtype)
    if fake_data:
      self._num_examples = 10000
      self.one_hot = one_hot
    else:
      assert images.shape[0] == labels.shape[0], (
          'images.shape: %s labels.shape: %s' % (images.shape, labels.shape))
      self._num_examples = images.shape[0]

      # Convert shape from [num examples, rows, columns, depth]
      # to [num examples, rows*columns] (assuming depth == 1)
      if reshape:
        assert images.shape[3] == 3
        images = images.reshape(images.shape[0],
                                images.shape[1] * images.shape[2],images.shape[3])
      if dtype == dtypes.float32:
        # Convert from [0, 255] -> [0.0, 1.0].
        images = images.astype(numpy.float32)
        images = numpy.multiply(images, 1.0 / 255.0)
    self._images = images
    self._labels = labels
    self._epochs_completed = 0
    self._index_in_epoch = 0

  @property
  def images(self):
    return self._images

  @property
  def labels(self):
    return self._labels

  @property
  def num_examples(self):
    return self._num_examples

  @property
  def epochs_completed(self):
    return self._epochs_completed

  def next_batch(self, batch_size, fake_data=False, shuffle=True):
    """Return the next `batch_size` examples from this data set."""
    if fake_data:
      fake_image = [1] * 1024
      if self.one_hot:
        fake_label = [1] + [0] * 9
      else:
        fake_label = 0
      return [fake_image for _ in xrange(batch_size)], [
          fake_label for _ in xrange(batch_size)
      ]
    start = self._index_in_epoch
    # Shuffle for the first epoch
    if self._epochs_completed == 0 and start == 0 and shuffle:
      perm0 = numpy.arange(self._num_examples)
      numpy.random.shuffle(perm0)
      self._images = self.images[perm0]
      self._labels = self.labels[perm0]
    # Go to the next epoch
    if start + batch_size > self._num_examples:
      # Finished epoch
      self._epochs_completed += 1
      # Get the rest examples in this epoch
      rest_num_examples = self._num_examples - start
      images_rest_part = self._images[start:self._num_examples]
      labels_rest_part = self._labels[start:self._num_examples]
      # Shuffle the data
      if shuffle:
        perm = numpy.arange(self._num_examples)
        numpy.random.shuffle(perm)
        self._images = self.images[perm]
        self._labels = self.labels[perm]
      # Start next epoch
      start = 0
      self._index_in_epoch = batch_size - rest_num_examples
      end = self._index_in_epoch
      images_new_part = self._images[start:end]
      labels_new_part = self._labels[start:end]
      return numpy.concatenate(
          (images_rest_part, images_new_part), axis=0), numpy.concatenate(
              (labels_rest_part, labels_new_part), axis=0)
    else:
      self._index_in_epoch += batch_size
      end = self._index_in_epoch
      return self._images[start:end], self._labels[start:end]


@deprecated(None, 'Please use alternatives such as official/mnist/dataset.py'
            ' from tensorflow/models.')
def read_data_sets(train_dir,
                   fake_data=False,
                   one_hot=False,
                   dtype=dtypes.float32,
                   reshape=True,
                   validation_size=5000,
                   seed=None,
                   source_url=DEFAULT_SOURCE_URL):
  if fake_data:

    def fake():
      return DataSet(
          [], [], fake_data=True, one_hot=one_hot, dtype=dtype, seed=seed)

    train = fake()
    validation = fake()
    test = fake()
    return base.Datasets(train=train, validation=validation, test=test)

  if not source_url:  # empty string check
    source_url = DEFAULT_SOURCE_URL

  TRAIN_IMAGES = 'train-images-idx3-ubyte.gz'
  TRAIN_LABELS = 'train-labels-idx1-ubyte.gz'
  TEST_IMAGES = 't10k-images-idx3-ubyte.gz'
  TEST_LABELS = 't10k-labels-idx1-ubyte.gz'
  '''
  local_file = base.maybe_download(TRAIN_IMAGES, train_dir,
                                   source_url + TRAIN_IMAGES)
  with gfile.Open(local_file, 'rb') as f:
    train_images = extract_images(f)

  local_file = base.maybe_download(TRAIN_LABELS, train_dir,
                                   source_url + TRAIN_LABELS)
  with gfile.Open(local_file, 'rb') as f:
    train_labels = extract_labels(f, one_hot=one_hot)

  local_file = base.maybe_download(TEST_IMAGES, train_dir,
                                   source_url + TEST_IMAGES)
  with gfile.Open(local_file, 'rb') as f:
    test_images = extract_images(f)

  local_file = base.maybe_download(TEST_LABELS, train_dir,
                                   source_url + TEST_LABELS)
  with gfile.Open(local_file, 'rb') as f:
    test_labels = extract_labels(f, one_hot=one_hot)'''
  (train_images, train_labels), (test_images, test_labels) = cifar10.load_data() 

  if not 0 <= validation_size <= len(train_images):
    raise ValueError('Validation size should be between 0 and {}. Received: {}.'
                     .format(len(train_images), validation_size))

  validation_images = train_images[:validation_size]
  validation_labels = train_labels[:validation_size]
  train_images = train_images[validation_size:]
  train_labels = train_labels[validation_size:]

  options = dict(dtype=dtype, reshape=reshape, seed=seed)

  train = DataSet(train_images, train_labels, **options)
  validation = DataSet(validation_images, validation_labels, **options)
  test = DataSet(test_images, test_labels, **options)

  return base.Datasets(train=train, validation=validation, test=test)


@deprecated(None, 'Please use alternatives such as official/mnist/dataset.py'
            ' from tensorflow/models.')
def load_mnist(train_dir='MNIST-data'):
  return read_data_sets(train_dir)

In [0]:
#cifar = read_data_sets("/tmp/data/")
print(dir(cifar.train))
t,v =cifar.test.next_batch(1)
print(dir(t))

## ment

In [0]:
import math
def caps(
         u,                      #input #batch*6*6*(8*32=256)
         input_n_maps=32,
         input_n_dims=8,
         output_n_maps=10,
         output_n_dims=16,
         kernel_size=(8,8),
         stride=1,
         init_sigma = 0.1
         #routing=2
         ):
    
  input_width=int(u.get_shape()[3])  #6-ra tanul vagy nagyobb
  input_height=int(u.get_shape()[2]) #6-ra tanul vagy nagyobb
  assert input_height == 8
  out_height=math.floor((input_height-kernel_size[1])/stride)+1 #1 vagy tobb
  out_width=math.floor((input_width-kernel_size[0])/stride)+1   #1 vagy tobb
  assert out_width==1
  
  caps2_n_caps = output_n_maps*out_height*out_width #10*1*1
  
#u_hat kiszamolasa  
                                                         #batch*32*6*6*8
  u2_expanded = tf.expand_dims(u, -1,name="u2_expanded") #batch*32*6*6*8*1                           #####u2->u########
  assert u2_expanded.get_shape().as_list() == [None, 32, 8,8,16,1]
  
  u2_tile = tf.expand_dims(u2_expanded, 4,name="u2_tile") #batch*32*6*6*1*8*1
  assert u2_tile.get_shape().as_list() == [None, 32, 8,8,1,16,1]
  
  u2_tiled = tf.tile(u2_tile, [1, 1,1,1, caps2_n_caps, 1, 1],name="u2_tiled")  ##batch*32*6*6*10*8*1
  assert u2_tiled.get_shape().as_list() == [None, 32, 8,8,10,16,1]
  
  W_init = tf.random_normal(                                                    #1*32*6*6*10*16*8 
      shape=(1, input_n_maps,input_height,input_width, caps2_n_caps, output_n_dims, input_n_dims), 
      stddev=init_sigma, dtype=tf.float32, name="W_init")
  W = tf.Variable(W_init, name="W")
  assert W.get_shape().as_list() == [1, 32, 8,8,10,16,16]
  batch_size = tf.shape(X)[0]

  W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1, 1, 1], name="W_tiled")         ##batch*32*6*6*10*16*8
  assert W_tiled.get_shape().as_list() == [None, 32, 8,8,10,16,16]
 
  u2_hat = tf.matmul(W_tiled, u2_tiled,name="u2_hat") ###batch*32*6*6*10*16*1
  assert u2_hat.get_shape().as_list() == [None, 32, 8,8,10,16,1]
  
# 
  
  b2 = tf.zeros([batch_size, input_n_maps,input_height,input_width, caps2_n_caps, 1, 1],    #batch*32*6*6*10*1*1
                         dtype=np.float32, name="b2")
  assert b2.get_shape().as_list() == [None, 32, 8,8,10,1,1]
  
  c2 = tf.nn.softmax(b2, dim=4, name="c2") #batch*32*6*6*10*1*1 
  assert c2.get_shape().as_list() == [None, 32, 8,8,10,1,1]
  
  s2 = tf.multiply(c2, u2_hat,name="s2")  #broadcast #batch*32*6*6*10*16*1
  assert s2.get_shape().as_list() == [None, 32, 8,8,10,16,1]
  
  s2_sum = tf.reduce_sum(s2, axis=1, keep_dims=True,name="s2_sum")              #batch*1*6*6*10*16*1
  s2_sum1 = tf.reduce_sum(s2_sum, axis=2, keep_dims=True,name="s2_sum1")        #batch*1*1*6*10*16*1
  s2_sum2 = tf.reduce_sum(s2_sum1, axis=3, keep_dims=True,name="s2_sum2")       #batch*1*1*1*10*16*1
  assert s2_sum2.get_shape().as_list() == [None, 1,1,1,10,16,1]
  
  v2_round_1 = squash(s2_sum2, axis=-2,name="v2_round_1") #batch*1*1*1*10*16*1
  assert v2_round_1.get_shape().as_list() == [None, 1,1,1,10,16,1]
  
  #routing 2nd iteration

  v2_round_1_tiled = tf.tile(v2_round_1, [1, input_n_maps,1,1, 1, 1, 1],    ### batch*32*1*1*10*16*1
      name="v2_round_1_tiled")
  v2_round_1_tiled1 = tf.tile(v2_round_1_tiled, [1, 1,input_height,1, 1, 1, 1],   ### batch*32*6*1*10*16*1
      name="v2_round_1_tiled1")
  v2_round_1_tiled2 = tf.tile(v2_round_1_tiled1, [1, 1,1,input_width, 1, 1, 1],   ### batch*32*6*6*10*16*1
      name="v2_round_1_tiled2")
  assert v2_round_1_tiled2.get_shape().as_list() == [None, 32,8,8,10,16,1]

  agreement = tf.matmul(u2_hat, v2_round_1_tiled2,
                        transpose_a=True, name="agreement") #transpose utolso ketto dimenziot csereli -> batch*32*6*6*10*1*1
  assert agreement.get_shape().as_list() == [None, 32,8,8,10,1,1]
  
  b2_round_2 = tf.add(b2, agreement,name="b2_round_2")                        # batch 32,8,8,10,1,1
  assert b2_round_2.get_shape().as_list() == [None, 32,8,8,10,1,1]
  c2_round_2 = tf.nn.softmax(b2_round_2,dim=4,name="c2_round_2")              # batch 32,8,8,10,1,1
  assert c2_round_2.get_shape().as_list() == [None, 32,8,8,10,1,1]
  s2_round_2 = tf.multiply(c2_round_2,u2_hat,name="s2_round_2")               # batch 32,8,8,10,16,1]
  assert s2_round_2.get_shape().as_list() == [None, 32,8,8,10,16,1]
  
  s2_sum_round_2 = tf.reduce_sum(s2_round_2,axis=1, keep_dims=True,name="s2_sum_round_2")         # batch 1,8,8,10,16,1]
  s2_sum1_round_2 = tf.reduce_sum(s2_sum_round_2, axis=2, keep_dims=True,name="s2_sum1_round_2")  # batch 1,1,8,10,16,1]
  s2_sum2_round_2 = tf.reduce_sum(s2_sum1_round_2, axis=3, keep_dims=True,name="s2_sum2_round_2") # batch 1,1,1,10,16,1]
  assert s2_sum2_round_2.get_shape().as_list() == [None, 1,1,1,10,16,1]
  
  v2_round_2 = squash(s2_sum2_round_2,axis=-2,name="v2_round_2")                # batch 1,1,1,10,16,1]
  assert v2_round_2.get_shape().as_list() == [None, 1,1,1,10,16,1]
  
  caps2_output = tf.reshape(v2_round_2, shape=(-1,output_n_maps,1,1,output_n_dims), name="caps2_output") # batch 10,1,1,16] 
  assert caps2_output.get_shape().as_list() == [None, 10,1,1,16]
  return caps2_output

### N-dim sliding 2Dwindow

In [0]:
def kernel_tile(input, kernel=8, stride=1,input_n_maps=32,input_n_dims=8):
 
    input_shape = input.get_shape()  #batch*ih(nem kernel),iw(nem kernel),(imap*idim)
    tile_filter = np.zeros(shape=[kernel, kernel, input_shape[3],
                                  kernel * kernel], dtype=np.float32)
    for i in range(kernel):
        for j in range(kernel):
            tile_filter[i, j, :, i * kernel + j] = 1.0 # (3, 3, 512, 9)

    # (3, 3, 512, 9)  #ker=6*ker=6,256,(ker**2=channel_mult=36)
    tile_filter_op = tf.constant(tile_filter, dtype=tf.float32)

    # (?, 6, 6, 4608)  1,1,256*36=9216 #batch,oh,ow,(imap*idim)*chm
    output = tf.nn.depthwise_conv2d(input, tile_filter_op, strides=[
                                    1, stride, stride, 1], padding='VALID')

    output_shape = output.get_shape()
   
    output = tf.reshape(output, shape=[-1, output_shape[1], output_shape[2], input_shape[3], kernel * kernel]) #batch,6,6,512,9  #batch,oh,ow,(imap*idim),chm
    output = tf.transpose(output, perm=[0, 1, 2, 4, 3])  #batch,6,6,9,512    #batch,oh,ow,chm,(imap*idim)
    # (?, 6, 6, 9, 512)
    output = tf.reshape(output, shape=[-1, kernel * kernel * input_n_maps, input_n_dims]) #(batch*36,9*32(input_map)=288, 16) #batch*oh*ow,chm*imap,idim
    #conv11 = tf.reshape(conv114, shape=[-1, kernel * kernel * _32, pose, pose]) # (batch*36,9*32=288, 4,4)
    
    #(batch*36,288,16) #batch*1*1,36*32,8 #batch*oh*ow,chm*imap,idim  chm*imap=mennyi idim(kicsi kapszula) van egy window-bam
    return output


# M(<=N)-dimensional sliding window in N-dimension/patch extraction/~moving average

In [0]:
import numpy as np
from numpy.lib import stride_tricks

def cutup(data, blck, strd,caps1_n_dims,caps_n_of_maps):
    sh = np.array(data.shape)
    blck = np.asanyarray(blck)
    strd = np.asanyarray(strd)
    nbl = (sh - blck) // strd + 1
    
    strides = np.r_[data.strides * strd, data.strides]
    dims = np.r_[nbl, blck]
    data6 = stride_tricks.as_strided(data, strides=strides, shape=dims)
    #return data6.reshape([-1,caps_n_of_maps*blck[1]*blck[2],caps1_n_dims])
    return np.transpose(data6.reshape([-1,caps_n_of_maps,blck[1]*blck[2],caps1_n_dims]),(0,2,1,3)).reshape([-1,data6.shape[1],data6.shape[2],caps_n_of_maps*blck[1]*blck[2],caps1_n_dims])#.reshape(-1, *blck)

##Cosine distance Routing algorithm

In [0]:
 def routing_alg(v,u_hat,b,level): #v #batch*oh*ow,1,omap,odim,1; u_hat #batch*oh*ow,chm*imap,omap,odim,1 ; b #batch*oh*ow,chm*imap,omap,1,1; level rekurzio szint
  caps_num_i=u_hat.get_shape()[1]
  v_tiled = tf.tile(v, [1, caps_num_i,1, 1, 1],name="v_tiled")#batch*oh*ow,chm*imap,omap,odim,1  
      
  agreement = tf.matmul(u_hat, v_tiled,transpose_a=True, name="agreement") #transpose utolso ketto dimenziot csereli -> #batch*oh*ow,chm*imap,omap,1,1   
  
  b_round_2 = tf.add(b, agreement,name="b_round_2")   #batch*oh*ow,chm*imap,omap,1,1
  
  c_round_2 = tf.nn.softmax(b_round_2,dim=2,name="c_round_2")  #batch*oh*ow,chm*imap,omap,1,1
  
  s_round_2 = tf.multiply(c_round_2,u_hat,name="s_round_2")  #batch*oh*ow,chm*imap,omap,odim,1                                  
  
  s_sum_round_2 = tf.reduce_sum(s_round_2,axis=1, keep_dims=True,name="s_sum_round_2")  #batch*oh*ow,1,omap,odim,1
  
  v_round_2 = squash(s_sum_round_2,axis=-2,name="v_round_2")          #batch*oh*ow,1,omap,odim,1
  
  if level!=1:
    v2_round_2= routing_alg(v_round_2,u_hat,b_round_2,level-1)
  else:
    return v_round_2
  return v2_round_2

##EM Routing

In [0]:
(N, OH, OW, kh x kw x imap, omap, odim) (N, OH, OW, kh x kw x imap)input
em algoritmusnak output pose: (N, OH, OW, omap ,odim) via squeeze o_mean (24, 6, 6, 32, 16), # activation: (N, OH, OW, omap)

In [0]:
def mat_transform(input, output_cap_size,out_dim, size): # batch*oh*ow, imap_kh_kw, i_dim
    """Compute the vote.

    :param inputs: shape (size, 288, 16)
    :param output_cap_size: 32

    :return votes: (24, 5, 5, 3x3=9, 136)
    """
    ismatrixcaps=True
    caps_num_i = int(input.get_shape()[1]) # 288
    i_dim = int(input.get_shape()[2])
    if ismatrixcaps: #ha matrixcapsule
      output = tf.reshape(input, shape=[size, caps_num_i, 1, int(math.sqrt(i_dim)), int(math.sqrt(i_dim))]) # (size, 288, 1, 4, 4)

#     w = slim.variable('w', shape=[1, caps_num_i, output_cap_size, 4, 4], dtype=tf.float32,
#                       initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1.0)) # (1, 288, 32, 4, 4)
      w = tf.get_variable('w', shape=[1, caps_num_i, output_cap_size, int(math.sqrt(i_dim)), int(math.sqrt(i_dim))], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1.0))
      w = tf.tile(w, [size, 1, 1, 1, 1])  # (24, 288, 32, 4, 4)
      output = tf.tile(output, [1, 1, output_cap_size, 1, 1]) # (size, 288, 32, 4, 4)
    else: #vactorcapsule szamitasigenyesebb
      output = tf.reshape(input, shape=[size, caps_num_i, 1, 1, i_dim])
      w = tf.get_variable('w', shape=[1, caps_num_i, output_cap_size, i_dim,out_dim], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1.0))
      w = tf.tile(w, [size, 1, 1, 1,1])  # (24, 288, 32, 1,16)
      output = tf.tile(output, [1, 1, output_cap_size, 1,1]) # (size, 288, 32,16,1)

    votes = tf.matmul(output, w) # (24*6*6, 288, 32, 4, 4) (24, 288, 32, 16, 1)
    if ismatrixcaps:
      votes = tf.reshape(votes, [size, caps_num_i, output_cap_size, i_dim]) # (size*6*6, 288, 32, 16) batch*oh*ow, imap*kh*kw, i_dim
    else:
      votes = tf.reshape(votes, [size, caps_num_i, output_cap_size, out_dim]) # (size*6*6, 288, 32, 16) batch*oh*ow, imap*kh*kw, i_dim

    return votes

def coord_addition(votes, H, W):
    """Coordinate addition.

    :param votes: (24, 4, 4, 32, 10, 16)
    :param H, W: spaital height and width 4

    :return votes: (24, 4, 4, 32, 10, 16)
    """
    coordinate_offset_hh = tf.reshape(
      (tf.range(H, dtype=tf.float32) + 0.50) / H, [1, H, 1, 1, 1]
    )
    coordinate_offset_h0 = tf.constant(
      0.0, shape=[1, H, 1, 1, 1], dtype=tf.float32
    )
    
    #########talan hibat dob
    coordinate_offset_h = tf.stack(
      [coordinate_offset_hh, coordinate_offset_h0] + [coordinate_offset_h0 for _ in range(14)], axis=-1
    )  # (1, 4, 1, 1, 1, 16)

    coordinate_offset_ww = tf.reshape(
      (tf.range(W, dtype=tf.float32) + 0.50) / W, [1, 1, W, 1, 1]
    )
    coordinate_offset_w0 = tf.constant(
      0.0, shape=[1, 1, W, 1, 1], dtype=tf.float32
    )
    coordinate_offset_w = tf.stack(
      [coordinate_offset_w0, coordinate_offset_ww] + [coordinate_offset_w0 for _ in range(14)], axis=-1
    ) # (1, 1, 4, 1, 1, 16)

    # (24, 4, 4, 32, 10, 16)
    votes = votes + coordinate_offset_h + coordinate_offset_w

    return votes

In [0]:
import time
def matrix_capsules_em_routing(votes, i_activations, beta_v, beta_a, iterations, name='em_routing'):
  """The EM routing between input capsules (i) and output capsules (j).

  :param votes: (N, OH, OW, kh x kw x i, o, 4 x 4) = (24, 6, 6, 3x3*32=288, 32, 16)
  :param i_activation: activation from Level L (24, 6, 6, 288)
  :param beta_v: (1, 1, 1, 32)
  :param beta_a: (1, 1, 1, 32)
  :param iterations: number of iterations in EM routing, often 3.
  :param name: name.

  :return: (pose, activation) of output capsules.
  """

  votes_shape = votes.get_shape().as_list()

  with tf.variable_scope(name) as scope:

    # Match rr (routing assignment) shape, i_activations shape with votes shape for broadcasting in EM routing

    # rr: [3x3x32=288, 32, 1]
    # rr: routing matrix from each input capsule (i) to each output capsule (o)
    rr = tf.constant(
      1.0/votes_shape[-2], shape=votes_shape[-3:-1] + [1], dtype=tf.float32
    )

    # i_activations: expand_dims to (24, 6, 6, 288, 1, 1)
    i_activations = i_activations[..., tf.newaxis, tf.newaxis]

    # beta_v and beta_a: expand_dims to (1, 1, 1, 1, 32, 1]
    beta_v = beta_v[..., tf.newaxis, :, tf.newaxis]
    beta_a = beta_a[..., tf.newaxis, :, tf.newaxis]
    epsilon = 1e-9
    
    def m_step(rr, votes, i_activations, beta_v, beta_a, inverse_temperature):
      """The M-Step in EM Routing from input capsules i to output capsule j.
      i: input capsules (32)
      o: output capsules (32)
      h: 4x4 = 16
      output spatial dimension: 6x6
      :param rr: routing assignments. shape = (kh x kw x i, o, 1) =(3x3x32, 32, 1) = (288, 32, 1)
      :param votes. shape = (N, OH, OW, kh x kw x i, o, 4x4) = (24, 6, 6, 288, 32, 16)
      :param i_activations: input capsule activation (at Level L). (N, OH, OW, kh x kw x i, 1, 1) = (24, 6, 6, 288, 1, 1)
         with dimensions expanded to match votes for broadcasting.
      :param beta_v: Trainable parameters in computing cost (1, 1, 1, 1, 32, 1)
      :param beta_a: Trainable parameters in computing next level activation (1, 1, 1, 1, 32, 1)
      :param inverse_temperature: lambda, increase over each iteration by the caller.

      :return: (o_mean, o_stdv, o_activation)
      """
      
      rr_prime = rr * i_activations

      # rr_prime_sum: sum over all input capsule i
      rr_prime_sum = tf.reduce_sum(rr_prime, axis=-3, keep_dims=True, name='rr_prime_sum')

      # Mean of the output capsules: o_mean(24, 6, 6, 1, 32, 16)
      o_mean = tf.reduce_sum(
        rr_prime * votes, axis=-3, keep_dims=True
      ) / (rr_prime_sum + epsilon)

      # Standard deviation of the output capsule:  o_stdv (24, 6, 6, 1, 32, 16)
      o_stdv = tf.sqrt(
        tf.reduce_sum(
          rr_prime * tf.square(votes - o_mean), axis=-3, keep_dims=True
        ) / (rr_prime_sum + epsilon)
      )

      # o_cost_h: (24, 6, 6, 1, 32, 16)
      o_cost_h = (beta_v + tf.log(o_stdv + epsilon)) * rr_prime_sum

      # o_cost: (24, 6, 6, 1, 32, 1)
      # o_activations_cost = (24, 6, 6, 1, 32, 1)
      # For numeric stability.
      o_cost = tf.reduce_sum(o_cost_h, axis=-1, keep_dims=True)
      o_cost_mean = tf.reduce_mean(o_cost, axis=-2, keep_dims=True)
      o_cost_stdv = tf.sqrt(
        tf.reduce_sum(
          tf.square(o_cost - o_cost_mean), axis=-2, keep_dims=True
        ) / (o_cost.get_shape().as_list()[-2]+epsilon)
      )
      o_activations_cost = beta_a + (o_cost_mean - o_cost) / (o_cost_stdv + epsilon)

      # (24, 6, 6, 1, 32, 1)
      o_activations = tf.sigmoid(
        inverse_temperature * o_activations_cost
      )

      return o_mean, o_stdv, o_activations

    def e_step(o_mean, o_stdv, o_activations, votes):
      """The E-Step in EM Routing.

      :param o_mean: (24, 6, 6, 1, 32, 16)
      :param o_stdv: (24, 6, 6, 1, 32, 16)
      :param o_activations: (24, 6, 6, 1, 32, 1)
      :param votes: (24, 6, 6, 288, 32, 16)

      :return: rr
      """

      o_p_unit0 = - tf.reduce_sum(
        tf.square(votes - o_mean) / (2 * tf.square(o_stdv)+epsilon), axis=-1, keep_dims=True
      )

      o_p_unit2 = - tf.reduce_sum(
        tf.log(o_stdv + epsilon), axis=-1, keep_dims=True
      )

      # o_p is the probability density of the h-th component of the vote from i to j
      # (24, 6, 6, 1, 32, 16)
      o_p = o_p_unit0 + o_p_unit2

      # rr: (24, 6, 6, 288, 32, 1)
      zz = tf.log(o_activations + epsilon) + o_p
      rr = tf.nn.softmax(
        zz, dim=len(zz.get_shape().as_list())-2
      )

      return rr

    # inverse_temperature schedule (min, max)
    it_min = 1.0
    it_max = min(iterations, 3.0)
    for it in range(iterations):
      #time.sleep(5)
      
      inverse_temperature = it_min + (it_max - it_min) * it / max(1.0, iterations - 1.0)
      o_mean, o_stdv, o_activations = m_step(
        rr, votes, i_activations, beta_v, beta_a, inverse_temperature=inverse_temperature
      )
      if it < iterations - 1:
        rr = e_step(
          o_mean, o_stdv, o_activations, votes
        )

    # pose: (N, OH, OW, o ,4 x 4) via squeeze o_mean (24, 6, 6, 32, 16)
    poses = tf.squeeze(o_mean, axis=-3)

    # activation: (N, OH, OW, o) via squeeze o_activationis [24, 6, 6, 32]
    activations = tf.squeeze(o_activations, axis=[-3, -1])

  return poses, activations


### mexicoi srac kodja

In [0]:
import keras as K
def em_routing(self, V_ij, a_i,beta_v,beta_a,n_caps_out,routing_iters,kernel_size,spatial_size,batch_size,n_caps_in,pose_shape):
        def maximization(R_ij, V_ij, a_i, inv_temp):
            # R_ij: SHAPE=[b, s0', s1', k0*k1*I, O, 1] or [b, s0'*s1'*I, O, 1], TYPE= tensor, VALUE=weights assignment according to the activation probabilities CAUTION!!!! maybe reshape it into k, k, A, B, 1 .... before multiplication
            R_ij = R_ij * a_i
            # R_ij: SHAPE=[b, (s0', s1',) 1, O, 1] , TYPE= tensor, VALUE=sum over all input capsules i
            R_ij_sum = K.sum(R_ij, axis=-3, keepdims=True)
            # M_j: SAHPE=[b, (s0', s1',) 1, O, p0*p1], TYPE= tensor, VALUE= mean of capsule j
            M_j = K.sum(R_ij * V_ij, axis=-3, keepdims=True ) / R_ij_sum
            # stdv_j: SAHPE=[b, (s0', s1',) 1, O, p0*p1], TYPE= tensor, VALUE= standard deviation of capsule j
            stdv_j = K.sqrt(K.sum(R_ij_sum * tf.square(V_ij - M_j), axis=-3, keepdims=True) / R_ij_sum)
            # cost_j_h: SHAPE=[b, (s0', s1',) 1, O, p0*p1], TYPE= tensor, VALUE= expected energy of a capsule j
            cost_j_h = (beta_v + K.log(stdv_j + K.epsilon())) * R_ij_sum
            # cost_j: SHAPE=[b, (s0', s1',) 1, O, 1], TYPE= tensor, VALUE= expected energy
            cost_j = K.sum(cost_j_h, axis=-1, keepdims=True)
            # cost_j_mean: SHAPE=[b, (s0', s1',) 1, 1, 1], TYPE= tensor, VALUE= mean the expected energy over the output capsules
            cost_j_mean = K.mean(cost_j, axis=-2, keepdims=True)
            # cost_j_stdv: SHAPE=[b, (s0', s1',) 1, 1, 1], TYPE= tensor, VALUE= mean the expected energy
            cost_j_stdv = K.sqrt(K.sum(K.square(cost_j - cost_j_mean), axis=-2, keepdims=True) / n_caps_out)
            # a_j_cost: SHAPE=[b, (s0', s1',) 1, O, 1], TYPE= tensor, VALUE= cost of the activation of capsule j
            a_j_cost = beta_a + (cost_j_mean - cost_j) / (cost_j_stdv + K.epsilon())
            # a_j: SHAPE=[b, (s0', s1',) 1, O, 1], TYPE= tensor, VALUE= activation of capsule j
            a_j = tf.sigmoid(inv_temp * a_j_cost)
            # a_j: SHAPE=[b, (s0', s1',) O], TYPE= tensor, VALUE= squeezed activation of capsule j
            a_j = K.squeeze(K.squeeze(a_j, axis=-3), axis=-1)
            # M_j: SAHPE=[b, (s0', s1',) O, p0*p1], TYPE= tensor, VALUE=squeezed mean of capsule j (pose matrix)
            M_j = K.squeeze(M_j, axis=-3)
            # stdv_j: SAHPE=[b, (s0', s1',) O, p0*p1], TYPE= tensor, VALUE=squeezed standard deviation of capsule j
            stdv_j = K.squeeze(stdv_j, axis=-3)
            return M_j, stdv_j, a_j
        def estimation(M_j, stdv_j, V_ij, a_j):
            # M_j: SAHPE=[b, (s0', s1',) 1, O, p0*p1], TYPE= tensor, VALUE=squeezed mean of capsule j (pose matrix)
            M_j = K.expand_dims(M_j, -3)     
            # a_j: SAHPE=[b, (s0', s1',) 1, O, 1], TYPE= tensor, VALUE=squeezed mean of capsule j (pose matrix)
            a_j = K.expand_dims(K.expand_dims(a_j, -2), -1)
            # stdv_j: SAHPE=[b, (s0', s1',) 1, O, p0*p1], TYPE= tensor, VALUE=squeezed mean of capsule j (pose matrix)
            stdv_j = K.expand_dims(stdv_j, -3)
            # R_ij: SHAPE= [b, s0', s1', k0*k1*I, O, 1] or [b, s0'*s1'*I, O, 1], TYPE= tensor, VALUE= routing matrix
            a_j_p_j  = K.log(a_j + K.epsilon()) - K.sum(K.square(V_ij - M_j) /(2 * tf.square(stdv_j)), axis=-1, keepdims=True) - K.sum(tf.log(stdv_j + K.epsilon()), axis=-1, keepdims=True)
            # R_ij: SHAPE= [b, s0', s1', k0*k1*I, O, 1], TYPE= tensor, VALUE= activated routing matrix
            R_ij = tf.nn.softmax(a_j_p_j, dim=len(a_j_p_j.get_shape().as_list())-2)
            return R_ij
        if len(self.spatial_dim):
            # V_ij: SHAPE=[b, s0', s1', k0*k1*I, O, p0*p1], TYPE= tensor, VALUE= adapt the shape for computation
            V_ij = K.reshape(V_ij, shape=[batch_size, spatial_size[0], spatial_size[1], kernel_size[0]*kernel_size[1]*n_caps_in,n_caps_out, pose_shape[0]*pose_shape[1]])
            # ai: SHAPE=[b, s0', s1', k0*k1*I, 1, 1], TYPE= tensor, VALUE= expanded i activations
            a_i = K.expand_dims(K.expand_dims(a_i,-1),-1)
            # R_ij: SHAPE=[k0*k1*I, O, 1], TYPE= tensor, VALUE= routing assignment matrix from each input capsule (i) in L to each output capsule (j) in L+1 initilized with uniform distribution
            R_ij = K.constant(1.0/n_caps_out, shape=(kernel_size[0]*kernel_size[0]*n_caps_in, n_caps_out, 1))
        else:
            # V_ij: SHAPE=[b, s0'*s1'*I, O, p0*p1], TYPE= tensor, VALUE= adapt the shape for computation
            V_ij = K.reshape(V_ij, shape=[batch_size, spatial_size[0]*spatial_size[1]*n_caps_in, n_caps_out, pose_shape[0]*pose_shape[1]])
            # a_i: SHAPE=[b, s0'*s1'*I, 1, 1], TYPE= tensor, VALUE= reshape to standard form
            a_i = K.reshape(a_i, shape=[batch_size, spatial_size[0]*spatial_size[1]*n_caps_in, 1, 1])
            # R_ij: SHAPE=[s0'*s1'*I, O, 1], TYPE= tensor, VALUE= routing assignment matrix from each input capsule (i) in L to each output capsule (j) in L+1 initilized with uniform distribution
            R_ij = K.constant(1.0/n_caps_out, shape=(spatial_size[0]*spatial_size[0]*n_caps_in, n_caps_out, 1))
        for iter in range(routing_iters):
            # inv_temp: SHAPE=[1], TYPE= int, VALUE= Lambda: inverse temperature schedule (1, min(routing_iters, 3.0)-1)
            inv_temp = 1.0 + (min(routing_iters, 3.0) - 1.0) * iter / max(1.0, routing_iters - 1.0)
            # M_j: SAHPE=[b, (s0', s1',) O, p0*p1], TYPE= tensor, VALUE= mean of capsule j
            # stdv_j: SAHPE=[b, (s0', s1',) O, p0*p1], TYPE= tensor, VALUE= standard deviation of capsule j
            # a_j: SHAPE= [b, (s0', s1',) O], TYPE= tensor, VALUE= activation of capsule j
            M_j, stdv_j, a_j = maximization(R_ij, V_ij, a_i, inv_temp=inv_temp)
            # R_ij: SHAPE= [b, (s0', s1',) k0*k1*I, O, 1], TYPE= tensor, VALUE= activated routing matrix
            if iter < routing_iters - 1:
                R_ij = estimation(M_j, stdv_j, V_ij, a_j)
        # M_j: SHAPE=[b, (s0', s1',) O, p0, p1], TYPE= tensor, VALUE= reshape back to the standard norm
        M_j = K.reshape(M_j, shape=[batch_size, spatial_size[0], spatial_size[1], n_caps_out, pose_shape[0], pose_shape[1]])
        return M_j, a_j

### CAPS LAYER

In [0]:
import math
def caps(
         u_slide,                      #input #batch*6*6*(8*32=256)
         i_activation,
         input_height=8,
         input_width=8,
         input_n_maps=32,
         input_n_dims=16,
         output_n_maps=10,
         output_n_dims=16,
         kernel_size=8,
         stride=1,
         init_sigma = 0.1,
         routing=2
         ):
  
  out_height=math.floor((input_height-kernel_size)/stride)+1 #1 vagy tobb
  out_width=math.floor((input_width-kernel_size)/stride)+1   #1 vagy tobb
  assert out_width==1#ha output_caps_layer
  
#u_hat kiszamolasa     u#bemenet batch,ih,iw,imap*idim

  #u_slide = kernel_tile(u, kernel=kernel_size, stride=stride,input_n_maps=input_n_maps,input_n_dims=input_n_dims) #batch*oh*ow,chm*imap,idim
  
  #u_slide=fuggv(u_val=u.eval())
  caps_num_i = int(u_slide.get_shape()[1]) #  caps_num_i=input_n_maps*kernel_size*kernel_size
  u_reshaped = tf.reshape(u_slide, shape=[-1, caps_num_i, 1,input_n_dims, 1])   #batch*oh*ow,chm*imap,1,idim,1
  u_tiled = tf.tile(u_reshaped, [1, 1, output_n_maps, 1,1])                     #batch*oh*ow,chm*imap,omap,idim,1
                                                  
  W_init = tf.random_normal(shape=(1, caps_num_i,output_n_maps,  output_n_dims,input_n_dims),   #1,chm*imap,omap,odim,idim
                            stddev=init_sigma, dtype=tf.float32, name="W_init")
  W = tf.Variable(W_init, name="W")
  batch_size = tf.shape(X)[0]
  W_tiled = tf.tile(W, [batch_size*out_height*out_width, 1, 1, 1, 1], name="W_tiled")  #batch*oh*ow,chm*imap,omap,odim,idim
  
  u2_hat = tf.matmul(W_tiled, u_tiled,name="u2_hat")  #batch*oh*ow,chm*imap,omap,odim,1

##########################################first routing iteration#############################################################
  b2 = tf.zeros([batch_size*out_height*out_width, caps_num_i, output_n_maps, 1, 1], #batch*oh*ow,chm*imap,omap,1,1
                         dtype=np.float32, name="b2")
  
  c2 = tf.nn.softmax(b2, dim=2, name="c2") #batch*oh*ow,chm*imap,omap,1,1 #omap szerint
  
  s2 = tf.multiply(c2, u2_hat,name="s2")  #broadcast  #batch*oh*ow,chm*imap,omap,odim,1
  
  s2_sum = tf.reduce_sum(s2, axis=1, keep_dims=True,name="s2_sum")   #batch*oh*ow,1,omap,odim,1 #chm*imap szerint>:(, miert nem csak imap szerint->mert konvolucional is a szumma[] 1-től össz pixel(=kernel**2)-ig:kernel* pixel         #batch*1*10*16*1
  #itt lehetnek problemak mert nem szamoltak ow iwvel/lehet hogy nem zavar be
  
  v2_round_1 = squash(s2_sum, axis=-2,name="v2_round_1")  #odim szerint batch*oh*ow,1,omap,odim,1
##########################################routing 2nd iteration and so on##################################################
  v2_round_2=routing_alg(v2_round_1,u2_hat,b2,routing-1)
  caps2_output = tf.reshape(v2_round_2, shape=(-1,output_n_maps,output_n_dims), name="caps2_output") #batch*oh*ow,omap,odim
 
  return caps2_output

### Caps Layer for EM routing

EM kernel utan (?, 6, 6, 3x3=9, 32x16=512)(?, 6, 6, 9, 32) 
resh (?, 9x32=288, 16)  (?,6,6, 9x32=288, 1)

u_slide=?, caps1_n_mapsx9x9,caps1_n_dims
activation_slide=caps1_n_mapsx9x9
         mit kene kernelnek=
         mi kene bemenetnek

In [0]:
import math
from tensorflow.contrib.layers.python.layers import initializers

def caps_EM(
         inputs_poses,                      #input #batch*6*6*(8*32=256)
         inputs_activations,
#          input_height=8,
#          input_width=8,
#          input_n_maps=32,
#          input_n_dims=16,
         o_size=10, #output_n_maps
         out_dim=16, #output_n_dims
#          kernel_size=8,
#          init_sigma = 0.1,
         batch_size=48,
         name='conv_caps1',
         iterations=3
         ):
  #after kernel tile batch, oh, ow, imap*kh*kw, idim 
  
  oh = int(inputs_poses.get_shape()[1])
  ow = int(inputs_poses.get_shape()[2])
  imap_kh_kw=int(inputs_poses.get_shape()[3])
  i_dim=int(inputs_poses.get_shape()[4])
  # Reshape it for later operations
  inputs_poses = tf.reshape(inputs_poses, shape=[-1, imap_kh_kw, i_dim])  # (?, 9x32=288, 16)
  inputs_activations = tf.reshape(inputs_activations, shape=[-1, oh, ow, imap_kh_kw]) # (?, 6, 6, 9x32=288)

  with tf.variable_scope('votes') as scope:

      # Generate the votes by multiply it with the transformation matrices
      votes = mat_transform(inputs_poses, o_size,out_dim, size=batch_size*oh*ow)  # (864, 288, 32, 16)

      # Reshape the vote for EM routing
      votes_shape = votes.get_shape()
      votes = tf.reshape(votes, shape=[batch_size, oh, ow, votes_shape[-3], votes_shape[-2], votes_shape[-1]]) # (24, 6, 6, 288, 32, 16)
      #tf.logging.info(f"{name} votes shape: {votes.get_shape()}")
      if 1:
        votes = coord_addition(votes, oh, ow)
        
  with tf.variable_scope('routing') as scope:

      # beta_v and beta_a one for each output capsule: (1, 1, 1, 32)
      beta_v = tf.get_variable(
          name='beta_v', shape=[1, 1, 1, o_size], dtype=tf.float32,
          initializer=initializers.xavier_initializer()
      )
      beta_a = tf.get_variable(
          name='beta_a', shape=[1, 1, 1, o_size], dtype=tf.float32,
          initializer=initializers.xavier_initializer()
      )

      # Use EM routing to compute the pose and activation
      # votes (24, 6, 6, 3x3x32=288, 32, 16), inputs_activations (?, 6, 6, 288)
      # poses (24, 6, 6, 32, 16), activation (24, 6, 6, 32)
      poses, activations = matrix_capsules_em_routing(
          votes, inputs_activations, beta_v, beta_a, iterations, name='em_routing'
      )

      # Reshape it back to 4x4 pose matrix
################szerintem nem kell      
#       poses_shape = poses.get_shape()
#       # (24, 6, 6, 32, 4, 4)
#       poses = tf.reshape(
#           poses, [
#               poses_shape[0], poses_shape[1], poses_shape[2], poses_shape[3], pose_size, pose_size
#           ]
#       )

#   tf.logging.info(f"{name} pose shape: {poses.get_shape()}")
#   tf.logging.info(f"{name} activations shape: {activations.get_shape()}")

 
  return poses, activations #batch, oh, ow, omap, odim

In [0]:
tf.set_random_seed(None)
np.random.seed(43)

#Capsnet

In [0]:
def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

In [0]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data

# mnist = input_data.read_data_sets("/tmp/data/")
#cifar
mnist = read_data_sets("/tmp/data/")

X = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32, name="X")
regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
conv1_params = {
    "kernel_regularizer":tf.contrib.layers.l2_regularizer(0.1),
    "filters":256,# 64,
    "kernel_size": 9,
    "strides": 1,
    "padding": "valid", #0 padding, same 1
    "activation": tf.nn.relu,
    "kernel_regularizer":regularizer
}
conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)

#########################################primary capsule layer######################################
caps1_n_maps = 32
caps1_n_dims = 16 #16 idealis 0.0005 alphaval!

conv2_params = {
    "kernel_regularizer":tf.contrib.layers.l2_regularizer(0.1),#scope=None),
    "filters": caps1_n_maps * caps1_n_dims,  # 256 convolutional filters eredetivel
    "kernel_size": 9,
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.leaky_relu,
    "kernel_regularizer":regularizer
}

conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params) #(?, 6, 6, 256   ) #batch,ih,iw,imap*idim
initial =tf.initializers.random_uniform# tf.constant_initializer(0.001)
primary_activ_params = {
    "kernel_regularizer":tf.contrib.layers.l2_regularizer(0.1),#scope=None),
    "filters": caps1_n_maps,  # 32 convolutional filters eredetivel
    "kernel_size": 9,
    "strides": 2,
    "padding": "valid",
    "kernel_initializer":initial,
    "kernel_regularizer":regularizer,
    "activation": tf.nn.leaky_relu
}

primary_activation = tf.layers.conv2d(conv1, name="primary_activation", **primary_activ_params) #(?, 6, 6, 32   ) #batch,ih,iw,imap

height=int(conv2.get_shape()[1])
width=int(conv2.get_shape()[2])
stride_h=1
stride_w=1
assert height==8        
#u2 = squash(conv2, name="u2") #u2 is the same as v1  #batch,ih,iw,imap*idim batch*1152*8 b*6*6**(32*8)  b 8*8*(32*16)
kernel_h=height#batch,oh,ow,imap,1,kh,kw,idim
kernel_w=width

out_height=math.floor((height-kernel_h)/stride_h)+1 #1 vagy tobb
out_width=math.floor((width-kernel_w)/stride_w)+1   #1 vagy tobb
u_slide= tf.placeholder(shape=[None,out_height,out_width, caps1_n_maps*kernel_h*kernel_w,caps1_n_dims], dtype=tf.float32, name="u_slide")
activation_slide= tf.placeholder(shape=[None,out_height,out_width, caps1_n_maps*kernel_h*kernel_w,1], dtype=tf.float32, name="activation_slide")
################################################real capsule layer##############################################
caps_params = {
       
#          "input_height":height,
#          "input_width": width,
#          "input_n_maps":caps1_n_maps,
#          "input_n_dims":caps1_n_dims,
         "o_size":10,
         "out_dim":16, #vectorcapsnal lenyeges , matrixnal az input merete lesz barmilyen out_dim-re
#          "kernel_size":9,
#          "stride":1,
#          "init_sigma": 0.1,
         "iterations":2
}

caps2_output,caps2_output_akt =caps_EM(u_slide,activation_slide,**caps_params) #batch, oh, ow, omap, odim


Spread loss

In [0]:
def spread_loss(labels, activations, iterations_per_epoch, global_step, name):
    """Spread loss

    :param labels: (24, 10] in one-hot vector
    :param activations: [24, 10], activation for each class
    :param margin: increment from 0.2 to 0.9 during training

    :return: spread loss
    """

    # Margin schedule
    # Margin increase from 0.2 to 0.9 by an increment of 0.1 for every epoch
    
    margin = tf.train.piecewise_constant(
        tf.cast(global_step, dtype=tf.int32),
        boundaries=[
            (iterations_per_epoch * x) for x in range(1, 8)
        ],
        values=[
            x / 10.0 for x in range(2, 10)
        ]
    )

    activations_shape = activations.get_shape().as_list()

    with tf.variable_scope(name) as scope:
        # mask_t, mask_f Tensor (?, 10)
        mask_t = tf.equal(labels, 1)      # Mask for the true label
        mask_i = tf.equal(labels, 0)      # Mask for the non-true label

        # Activation for the true label
        # activations_t (?, 1)
        activations_t = tf.reshape(
            tf.boolean_mask(activations, mask_t), shape=(tf.shape(activations)[0], 1)
        )
        
        # Activation for the other classes
        # activations_i (?, 9)
        activations_i = tf.reshape(
            tf.boolean_mask(activations, mask_i), [tf.shape(activations)[0], activations_shape[1] - 1]
        )

        l = tf.reduce_sum(
            tf.square(
                tf.maximum(
                    0.0,
                    margin - (activations_t - activations_i)
                )
            )
        )
        tf.losses.add_loss(l)

        return l


Margin Loss and reconstruction loss

In [0]:
out_map=caps2_output.get_shape()[3] #10
out_dim=caps2_output.get_shape()[4] #16
caps2_output_akt=tf.squeeze(caps2_output_akt,[1,2])
#####################################################Margin Loss##################################################
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")
T = tf.one_hot(y, depth=10, name="T")

caps2_output_norm = safe_norm(caps2_output, axis=-1, keep_dims=False,            #batch,10,16->1
                              name="caps2_output_norm")
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),       #batch,10
                              name="present_error_raw")

absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")

L = tf.add(T * present_error_raw, lambda_ * (1.0 - T) * absent_error_raw,name="L")#batch,10
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss") #scalar

#Masking for reconstruction

#y_proba = safe_norm(caps2_output, axis=-1, name="y_proba") #batch,1,1,10
#y_proba = safe_norm(caps2_output_akt, axis=-1, name="y_proba") #batch,1,1,10
y_pred= tf.argmax(caps2_output_akt, axis=-1, name="y_pred") #batch
#print(y_pred)
mask_with_labels = tf.placeholder_with_default(False, shape=(),name="mask_with_labels")

reconstruction_targets = tf.cond(mask_with_labels,  # condition  mit rekonstrualtatunk tanitas alatt ->amit josol, vagy amit kellett volna
                                 lambda: y,  # if True
                                 lambda: y_pred,  # if False
                                 name="reconstruction_targets")

reconstruction_mask = tf.one_hot(reconstruction_targets,
                                 depth=out_map,
                                 name="reconstruction_mask")

reconstruction_mask_reshaped = tf.reshape(
    reconstruction_mask, [-1,1,1, out_map, 1], #megforgatva ##########################baj szokott lenni
    name="reconstruction_mask_reshaped")


caps2_output_masked = tf.multiply(
    caps2_output, reconstruction_mask_reshaped,
    name="caps2_output_masked")

decoder_input = tf.reshape(caps2_output_masked,
                           [-1, out_map * out_dim],   #batch*160
                           name="decoder_input")

#Reconstruction FFNN layer

n_hidden1 = 512*3
n_hidden2 = 1024*3
n_output = 32 * 32*3

with tf.name_scope("decoder"):
    hidden1 = tf.layers.dense(decoder_input, n_hidden1,
                              activation=tf.nn.relu,
                              name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu,
                              name="hidden2")
    decoder_output0 = tf.layers.dense(hidden2, n_output,
                                     activation=tf.nn.sigmoid,
                                     name="decoder_output0")

decoder_output=tf.reshape(decoder_output0,(-1,32*32,3))
X_flat = tf.reshape(X, [-1, n_output], name="X_flat")
X_flat=tf.reshape(X_flat,(-1,32*32,3))

squared_difference = tf.square(X_flat - decoder_output,
                               name="squared_difference")
reconstruction_loss = tf.reduce_mean(squared_difference,
                                     name="reconstruction_loss")

alpha = 0.0005 #0.0005
n_epochs = 1
batch_size = 48
n_iterations_per_epoch = mnist.train.num_examples // batch_size
n_iterations_validation = mnist.validation.num_examples // batch_size
n_iterations_test = mnist.test.num_examples // batch_size
global_step = tf.train.get_or_create_global_step()
loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")
loss_tr=spread_loss(T,caps2_output_akt,n_iterations_per_epoch,global_step,'spread_loss')
loss_v=spread_loss(T,caps2_output_akt,n_iterations_validation,global_step,'spread_loss')
loss_te=spread_loss(T,caps2_output_akt,n_iterations_test,global_step,'spread_loss')
loss_tr=tf.add(loss_tr,alpha * reconstruction_loss, name="loss")
loss_v=tf.add(loss_v,alpha * reconstruction_loss, name="loss")
loss_te=tf.add(loss_te,alpha * reconstruction_loss, name="loss")

correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss_tr, name="training_op")

init = tf.global_variables_initializer()
init1 = tf.global_variables_initializer()
saver = tf.train.Saver()


best_loss_val = str(np.infty)
#checkpoint_path = "./my_capsule_network"
checkpoint_path = "./drive/mentes/tempg/"
checkpoint_path_trainer = "./drive/mentes/trainerg/"

In [0]:
n_epochs = 30
best_loss_val = str(np.infty)

In [0]:
init = tf.global_variables_initializer()
#init1 = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 30
batch_size = 48


n_iterations_per_epoch = mnist.train.num_examples // batch_size
n_iterations_validation = mnist.validation.num_examples // batch_size
best_loss_val = str(np.infty)
checkpoint_path = "./my_capsule_network/"
# checkpoint_path = "./drive/mentes/tempg/"
# checkpoint_path_trainer = "./drive/mentes/trainerg/"

# Training and validation of models

In [0]:
!pip install tensorboardcolab
from tensorboardcolab import *
n_epochs = 30
best_loss_val = str(np.infty)

!rm -r ./Graph
tbc = TensorBoardColab(startup_waiting_time=0)
writer = tbc.get_writer()

In [0]:
tbc.close()

In [0]:
!mkdir my_capsule_network

In [0]:
!echo "Hello, world." >./my_capsule_network/constants.txt

In [0]:
!cat ./my_capsule_network/constants.txt

In [0]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# C=np.array([[1,   0,   9  ],
#               [0,   3.0, 0],
#               [8,   0,   1  ]])
# classes=np.arange(10)
def plot_confusion_matrix(cm, jaj,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(3)
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.axis("off")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
#plot_confusion_matrix(C.astype(int),classes)

In [0]:
def tf_sess(s_type,batch_size,train_acc,train_loss):
  
  #X_batch, y_batch = exec('mnist.'+s_type+'.next_batch(batch_size)') TODO valahogy ezt implementalni
  
  #X_batch=np.array(np.zeros((batch_size, 32, 32, 3)))
  #y_batch=np.array(np.zeros(batch_size))

  X_batch=[]
  y_batch=[]

  if s_type=='train':
    X_batch, y_batch = mnist.train.next_batch(batch_size)
    actual_batch=X_batch.shape[0]
    #print(actual_batch)
    loss=loss_tr
  if s_type=='validation':
    X_batch, y_batch = mnist.validation.next_batch(batch_size)
    loss=loss_v
  if s_type=='test':
    X_batch, y_batch = mnist.test.next_batch(batch_size)
    loss=loss_te
  # Run the training operation and measure the loss:

  u_val,act_val = sess.run(
      [conv2,primary_activation], feed_dict={X: X_batch.reshape([-1, 32, 32, 3])}) #nem szep jobb lenne a capslayeren belul valamit csinalni
  #print(np.asarray(u_val).size)
  a=time.time()

  data=np.asarray(u_val).reshape([-1,height,width,caps1_n_dims*caps1_n_maps])
  data_akt=np.asarray(act_val).reshape([-1,height,width,caps1_n_maps])
#   if iteration>1:  #fuggvenyen belul nem mukodik mert egy iteracio utan torlodik
#     assert not np.array_equal(data1,data)
#   data1=data

  u_tiled= cutup(data,(1,kernel_h,kernel_w,caps1_n_dims),(1,stride_h,stride_w,caps1_n_dims),caps1_n_dims,caps1_n_maps)
  activation_tiled=cutup(data_akt,(1,kernel_h,kernel_w,1),(1,stride_h,stride_w,1),1,caps1_n_maps)
#             with tf.Session() as inner_sess:
#               init1.run()
  if s_type=='train':
    _, loss_train,acc_train,y_pred_,y_,ta,tl = sess.run(
        [training_op, loss, accuracy,y_pred,y,train_acc,train_loss],
        feed_dict={X: X_batch.reshape([-1, 32, 32, 3]),
                   u_slide: u_tiled,
                   activation_slide:activation_tiled,
                   mask_with_labels: mask,
                   y: np.reshape(y_batch,(-1))
                   })
    return loss_train,acc_train,y_pred_,y_,actual_batch,ta,tl
  else:
    loss_val_test,acc_val_test,y_,y_pred_,tva,tvl = sess.run(
        [loss, accuracy,y,y_pred,train_acc,train_loss],
        feed_dict={X: X_batch.reshape([-1, 32, 32, 3]),
                   u_slide: u_tiled,
                   activation_slide:activation_tiled,
                   mask_with_labels: mask,
                   y: np.reshape(y_batch,(-1))
                   })
    if s_type== "test":
      return loss_val_test, acc_val_test,y_,y_pred_,tva,tvl
    else:
      return loss_val_test, acc_val_test,tva,tvl

In [0]:
  print(best_loss_val)
  import time
  t=time.time()
  restore_checkpoint = False
  trainer_start = True
  mask=False
  
  eddig=0
  with tf.Session() as sess:
    if restore_checkpoint and (tf.train.checkpoint_exists(checkpoint_path) or trainer_start) :
      if trainer_start:
        saver.restore(sess, checkpoint_path_trainer)
        f = open(checkpoint_path_trainer+"constants.txt", 'r')
        best_loss_val=f.read()
        f.close()
      else:
        saver.restore(sess, checkpoint_path)
        f = open(checkpoint_path+"constants.txt", 'r')
        best_loss_val=f.read()
        f.close()
    else:
        init.run()
    '''4 db tab/space'''
    print(best_loss_val)
    
    train_acc=tf.summary.scalar("train_accuracy", accuracy)
    train_loss=tf.summary.scalar("train_loss", loss_tr)
    val_acc=tf.summary.scalar("validation_accuracy", accuracy)
    val_loss=tf.summary.scalar("validation_loss", loss_v)
    test_acc=tf.summary.scalar("test_accuracy", accuracy)
    test_loss=tf.summary.scalar("test_loss", loss_te)
    classes=[0,1,2,3,4,5,6,7,8,9]
    C2=np.zeros((10,10))
    for epoch in range(n_epochs):
        for iteration in range(0, n_iterations_per_epoch ):
   #         
            loss_train, acc_train,y_pred_train,y_train,actual_batch,ta,tl=tf_sess('train', batch_size,train_acc,train_loss)
            #lehet h u2 kell lehet hogy inneren belul kell
            #if iteration==n_iterations_per_epoch:
              #print(actual_batch)
          
            writer.add_summary(ta,(iteration+(epoch+eddig)*n_iterations_per_epoch))
            writer.add_summary(tl,(iteration+(epoch+eddig)*n_iterations_per_epoch))
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}  Accuracy_training: {:.4f}%".format(
                iteration+1, n_iterations_per_epoch+1,
                (iteration+1) * 100 / (n_iterations_per_epoch+1),
                loss_train,
                acc_train*100),
                end="")
            #print("\n")
            #print(y_train)
            #print(y_pred_train)
            #print(str((time.time()-a))+" sec")
        # At the end of each epoch,
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []
        x=range(100)
        for iteration in range(0, n_iterations_validation ):
          
    #       '
            loss_val,acc_val,va,vl=tf_sess('validation',batch_size,val_acc,val_loss)
            
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            
            writer.add_summary(va,(iteration+(epoch+eddig)*n_iterations_validation))
            writer.add_summary(vl,(iteration+(epoch+eddig)*n_iterations_validation))
            print("\rEvaluating!!! the model: {}/{} ({:.1f}%)".format(
                iteration, n_iterations_validation,
                iteration * 100 / n_iterations_validation),
                end=" " * 10)
        #plt.plot(x, acc_vals, label="acc_vals")
        #plt.plot(x, loss_vals, label="loss_vals")
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
            epoch + 1, acc_val * 100, loss_val,
            " (improved)" if loss_val < float(best_loss_val) else ""))
        
        n_iterations_test = mnist.test.num_examples // batch_size
        loss_tests = []
        acc_tests = []
        for iteration in range(0, n_iterations_test):
   #         
          loss_test,acc_test,y_,y_pred_,tsa,tsl=tf_sess('test',batch_size,test_acc,test_loss)
          loss_tests.append(loss_test)
          acc_tests.append(acc_test)
          if epoch == n_epochs-1:
            c2=confusion_matrix(y_,y_pred_)
            if c2.shape[0]==C2.shape[0]:
              C2=np.add(C2,c2)
            else:
  #             print("Rossz")
  #             print(y_pred_,"y_pred_")
  #             print(y_,"y_")
  #             print(c2)
              for i in range (0,10):
                if i not in y_:
                  c2=np.insert(c2, i, 0, axis=1)
                  c2=np.insert(c2, i, 0, axis=0)
  #             print(c2)
              C2=np.add(C2,c2)
            
          writer.add_summary(tsa,(iteration+(epoch+eddig)*n_iterations_test))
          writer.add_summary(tsl,(iteration+(epoch+eddig)*n_iterations_test))
          print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
            iteration, n_iterations_test,
            iteration * 100 / n_iterations_test),
            end=" " * 10)
        loss_test = np.mean(loss_tests)
        acc_test = np.mean(acc_tests)
        print("\rFinal test accuracy: {:.4f}%  Loss: {:.6f}".format(
            acc_test * 100, loss_test))
        
        # And save the model if it improved:
        print(type(loss_val))
        if loss_val < float(best_loss_val):
            print(str(checkpoint_path)+" start")
            f = open(checkpoint_path+"constants.txt", 'w')
            best_loss_val = str(loss_val)
            f.write(best_loss_val)
            save_path = saver.save(sess, checkpoint_path)
        if epoch==n_epochs-1:
          plot_confusion_matrix(C2.astype(int),classes)
    f.close()
#     print(str(checkpoint_path_trainer)+" start")
#     f1 = open(checkpoint_path_trainer+"constants.txt", 'w')
#     #best_loss_val = str(loss_val)
#     f1.write(best_loss_val)
#     save_path = saver.save(sess, checkpoint_path_trainer)
#     f1.close()
print(str((time.time()-t)/60)+" minutes")

#TODO validationra is megcsinalni az inner sessiont

### enforced saving

In [0]:
   with tf.Session() as sess: 
    init.run()
    if loss_val < float(best_loss_val):
            print(str(checkpoint_path)+" start")
            f = open(checkpoint_path+"constants.txt", 'w')
            best_loss_val = str(loss_val)
            f.write(best_loss_val)
            save_path = saver.save(sess, checkpoint_path)
            f.close()
    print(str(checkpoint_path_trainer)+" start")
    f1 = open(checkpoint_path_trainer+"constants.txt", 'w')
    #best_loss_val = str(loss_val)
    f1.write(best_loss_val)
    save_path = saver.save(sess, checkpoint_path_trainer)
    f1.close()

In [0]:
checkpoint_path = "./drive/mentes/tempg/"
checkpoint_path_trainer = "./drive/mentes/trainerg/"

In [0]:
!ls drive/mentes/temp

#Test accuracy and loss of best model

In [0]:
n_iterations_test = mnist.test.num_examples // batch_size

with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    
    loss_tests = []
    acc_tests = []
    for iteration in range(1, n_iterations_test + 1):
        X_batch, y_batch = mnist.test.next_batch(batch_size)
        loss_test, acc_test = sess.run(
            [loss, accuracy],
            feed_dict={X: X_batch.reshape([-1, 32, 32, 3]),
                       y: np.reshape(y_batch,(-1))})
        loss_tests.append(loss_test)
        acc_tests.append(acc_test)
        print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
            iteration, n_iterations_test,
            iteration * 100 / n_iterations_test),
            end=" " * 10)
    loss_test = np.mean(loss_tests)
    acc_test = np.mean(acc_tests)
    print("\rFinal test accuracy: {:.4f}%  Loss: {:.6f}".format(
        acc_test * 100, loss_test))
    !free -g
    
#TODO megcsinalni belso sessiont gondolom lentebb is

In [0]:
!ls ./drive/mentes/mnist/

#Visual

##Kernels

In [0]:
imageToUse = mnist.train.images[2] #10epoch-15,1epoch-67
print(imageToUse.shape)
plt.axis('off')
imageToUse=np.reshape(imageToUse,[32,32,3])
#imageToUse=np.transpose(imageToUse)
plt.imshow(imageToUse, interpolation="none", cmap="gray")

In [0]:
def getActivations(layer,stimuli):
    stimuli=np.reshape(stimuli,[-1,32,32,3])#,order='F')
    #stimuli = np.swapaxes(stimuli,axis1=1,axis2=1)
    units = sess.run(layer,feed_dict={X:stimuli,keep_prob:1.0})
    return units   
    
def plotNNFilter(units):
    filters = units.shape[3]
    print(units.shape)
    plt.figure(1,facecolor=(0.1, 0.2, 0.5, 0.3), figsize=(12,12))
    n_columns = 10 #ha ez a dimenzio es nem a n_maps akkor plotton sorok felelnek meg egy kapszulanak
    n_rows = math.ceil(filters / n_columns) + 1
    #units=np.swapaxes(np.asarray(units),1,2) #kell mert valamiert megforgatja->2.sor masolt kikommentelt resz miatt volt
    print(type(units[0].shape))
    mini=np.infty
    maxi=0
#     for i in range(units):
#       if max(i)>maxi:
#         maxi=max(i)
#     if min(i)<mini:
#         mini=min(i)
    for i in range(filters):
        
        plt.subplot(n_rows, n_columns, i+1)
        #plt.title('Filter ' + str(i))
        #plt.title(str(i))
        #plt.title(str(units[0,:,:,i]))
        plt.imshow(units[0,:,:,i], interpolation="nearest", cmap="brg")#,vmin=0, vmax=0.01)
        plt.axis('off')
        #plt.colorbar()
#         if i==10:
#           break
    #print(units[0,:,:,219])
    plt.show()

In [0]:
type(imageToUse)
imageToUse.shape
np.reshape(imageToUse,[-1,32,32,3],order='F').shape
np.swapaxes(np.reshape(imageToUse,[-1,32,32,3],order='F'),2,3).shape

In [0]:
with tf.Session() as sess:
  saver.restore(sess, checkpoint_path)
  #d=np.array(y_proba.eval({X:np.reshape(imageToUse,[-1,32,32,3])}))
  d=np.array(conv1.eval({X:np.reshape(imageToUse,[-1,32,32,3])}))
  print(d)
  print(np.argmax(d))
#   d2=tf.reshape(caps2_output, shape=(-1,1,1,160), name="caps2_output2")
#   plotNNFilter(d2)

In [0]:
keep_prob = tf.placeholder("float")
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
#     #caps2_output2=tf.reshape(caps2_output, shape=(-1,1,1,160), name="caps2_output2")
#     caps2_output=tf.reshape(y_proba, shape=(-1,1,1,10), name="caps2_output2")
#     plotNNFilter(getActivations(caps2_output,imageToUse))
    
    #plt.imshow(np.reshape(imageToUse,[32,32,3]), interpolation="none", cmap="gray") #allitani scale-t
    plotNNFilter(getActivations(conv1,imageToUse))
    
    print("   plane      auto     bird      cat      deer     dog     frog    horse    ship   truck")

sum

In [0]:
keep_prob = tf.placeholder("float")
import math
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    #tf.initialize_all_variables().run()
    
    sum=np.zeros((1,20,20,256))
    
    for i in range (0,mnist.test.images.shape[0]-1):
      imageToUse = mnist.test.images[i]
      sum =sum + np.abs(getActivations(conv1,imageToUse))#sajat
      print("\r{:.2f}".format(i/(mnist.test.images.shape[0]-1)*100)+'%',end="")
    print(" Finished")
    plotNNFilter(sum)
    

#Reconstruction and tweak

In [0]:
n_samples = 48 #455
mask=True
sample_images = mnist.train.images[:n_samples].reshape([-1, 32, 32, 3])
labelz = mnist.train.labels[:n_samples]

with tf.Session() as sess:
    #labelz = mnist.train.labels[:n_samples]
    saver.restore(sess, checkpoint_path_trainer)
    labelz = mnist.train.labels[:n_samples]
    print(np.reshape(labelz,(-1)))
    
    
    u_val,act_val = sess.run([conv2,primary_activation], feed_dict={X: sample_images.reshape([-1, 32, 32, 3])}) #nem szep jobb lenne a capslayeren belul valamit csinalni
                              #print(np.asarray(u_val).size)
    #a=time.time()

    data=np.asarray(u_val).reshape([-1,height,width,caps1_n_dims*caps1_n_maps])
    data_akt=np.asarray(act_val).reshape([-1,height,width,caps1_n_maps])
#     if iteration>1:
#       assert not np.array_equal(data1,data)
#     data1=data

    u_tiled= cutup(data,(1,kernel_h,kernel_w,caps1_n_dims),(1,stride_h,stride_w,caps1_n_dims),caps1_n_dims,caps1_n_maps)
    activation_tiled=cutup(data_akt,(1,kernel_h,kernel_w,1),(1,stride_h,stride_w,1),1,caps1_n_maps)
    
    
    caps2_output_value, decoder_output_value, y_pred_value = sess.run(
              [caps2_output, decoder_output, y_pred],
              feed_dict={X: sample_images.reshape([-1, 32, 32, 3]),
                           u_slide: u_tiled,
                           activation_slide:activation_tiled,
                           mask_with_labels: mask,
                           y: np.reshape(labelz,(-1))
                           })
                         #y: np.array([], dtype=np.int64)})
    print(y_pred_value)                   
sample_images = sample_images.reshape(-1, 32, 32,3)
reconstructions = decoder_output_value.reshape([-1, 32,32,3])

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    plt.imshow(sample_images[index], cmap="binary_r")
    plt.title("Eredeti címke:" + str(mnist.train.labels[index]))
    plt.axis("off")

plt.show()
#reconstructions[0]=np.zeros([28, 28, 1])
plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    plt.title("Rekonstruált \n választott:" + str(y_pred_value[index]))
    plt.imshow(reconstructions[index], cmap="binary_r")
    plt.axis("off")
#print(reconstructions[0])
plt.show()

In [0]:
reconstructions


In [0]:
def tweak_pose_parameters(output_vectors, min=-0.5, max=0.5, n_steps=11):
    out_dim=output_vectors.shape[2]
    out_map=output_vectors.shape[1]
    steps = np.linspace(min, max, n_steps)  # -0.25, -0.15, ..., +0.25
    pose_parameters = np.arange(out_dim)  # 0, 1, ..., 15
    tweaks = np.zeros([out_dim, n_steps, 1, 1, out_dim])
    tweaks[pose_parameters, :, 0, 0, pose_parameters] = steps    
    output_vectors_expanded = output_vectors[np.newaxis, np.newaxis] #(1, 1, 10, 10,  16)
    return tweaks + output_vectors_expanded

In [0]:
n_steps = 11

tweaked_vectors = tweak_pose_parameters(caps2_output_value, n_steps=n_steps)#(16, 11, 10, 20, 16)?
print(tweaked_vectors.shape)
#tweaked_vectors =tweaks = np.zeros([16, n_steps, 1, 1, 16]) + caps2_output_value[np.newaxis, np.newaxis]

tweaked_vectors_reshaped = tweaked_vectors.reshape(
    [-1, out_map, out_dim])

tweak_labels = np.tile(mnist.train.labels[:n_samples], out_dim * n_steps)
tweak_labels_reshaped=np.reshape(tweak_labels,(n_samples,out_dim*n_steps))#sajat
tweak_labels_tpd=np.transpose(tweak_labels_reshaped)#sajat
#print(tweak_labels)
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path_trainer)
    decoder_output_value2 , y_pred_V = sess.run(
        [decoder_output, y_pred ],
        feed_dict={caps2_output: tweaked_vectors_reshaped,
                   mask_with_labels: False,
                   y: np.reshape(tweak_labels_tpd,(-1))})
    #print(np.tile(np.reshape(y_pred_V[:n_samples],(-1,1)), 16 * n_steps))
    #print(np.reshape(tweak_labels,(-1)))#n_steps = 1 nel ellenoriztem vele mask_with_labels verzioit
    #print(y_pred_V)#n_steps = 1 nel ellenoriztem vele mask_with_labels verzioit
    

In [0]:
tweak_reconstructions = decoder_output_value2.reshape(
    [out_dim, n_steps, n_samples,32, 32,3])
#tweak_reconstructions[0,5,0]=np.zeros([28, 28, 1])

i=16
assert i <= out_dim
for dim in range(i):
    print("Tweaking output dimension #{}".format(dim))
    plt.figure(figsize=(n_steps / 1.2, n_samples / 1.5))
    
    for col in range(n_steps):    
        for row in range (n_samples):
            plt.subplot(n_samples, n_steps, row * n_steps + col + 1)
            plt.imshow(tweak_reconstructions[dim, col, row], cmap="binary")
            plt.axis("off")
    plt.show()
# for i in range(tweak_reconstructions.shape[0]):
#   for j in range(tweak_reconstructions.shape[1]):
#     for k in range (tweak_reconstructions.shape[2]):
#       for l in range (tweak_reconstructions.shape[3]):
#         for m in range (tweak_reconstructions.shape[4]):
#           for n in range (tweak_reconstructions.shape[5]):
#             if abs(tweak_reconstructions[i,j,k,l,m,n]-0.64136815)<1e-4 :
#               print(i,j,k,l,m,n)